In [59]:
import cv2
import numpy as np
import mediapipe as mp
import os
import pandas as pd

In [60]:
MP_Pose = mp.solutions.pose
Pose = MP_Pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7 )

Drawing = mp.solutions.drawing_utils
Drawing_Spec_Landmarks = mp.solutions.drawing_utils.DrawingSpec(thickness=2, circle_radius=8, color=(255, 0, 127))
Drawing_Spec_Connections = mp.solutions.drawing_utils.DrawingSpec(thickness=4, color=(0, 0, 102))
#Drawing_Spec = Drawing.DrawingSpec(thickness=2, circle_radius=4, color=(0,255,127)) 

## Angle Calculation

In [61]:
def Calculate_Angle(Point_1, Point_2, Point_3):
    Point_1 = np.array(Point_1) 
    Point_2 = np.array(Point_2)
    Point_3 = np.array(Point_3) 
    
    radians = np.arctan2(Point_3[1]-Point_2[1], Point_3[0]-Point_2[0]) - np.arctan2(Point_1[1]-Point_2[1], Point_1[0]-Point_2[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


## Main Angles Definition

In [62]:
Main_Angles = {
    
    "L_Neck"     : [0,11,12,0]  , 
    "R_Neck"     : [0,12,11,1]  , 
    "L_Shoulder" : [13,11,23,2] , 
    "R_Shoulder" : [14,12,24,3] , 
    "L_Elbow"    : [11,13,15,4] , 
    "R_Elbow"    : [12,14,16,5] ,
    "L_Wrist"    : [13,15,19,6] , 
    "R_Wrist"    : [14,16,20,7] ,
    "L_Hip"      : [11,23,25,8] , 
    "R_Hip"      : [12,24,26,9] ,
    "L_Knee"     : [23,25,27,10] ,
    "R_Knee"     : [24,26,28,11] ,
    "L_Ankle"    : [25,27,31,12] ,
    "R_Ankle"    : [26,28,32,13] 
}


![Body Landmarks ](Assets//Body_LandMarks.png)

## Draw Angles 

In [63]:
def Draw_Angles( Counter , image , L_Marks ) :
    
    height, width, _ = image.shape
    for Key, Val in Main_Angles.items():
        
        Angle = int(Calculate_Angle([L_Marks[Val[0]].x, L_Marks[Val[0]].y],
                                    [L_Marks[Val[1]].x, L_Marks[Val[1]].y],
                                    [L_Marks[Val[2]].x, L_Marks[Val[2]].y]))
        
        txt = tuple(np.multiply([L_Marks[Val[1]].x, L_Marks[Val[1]].y],
                                [width,height if (Key != "L_Neck" and Key != "R_Neck") else height -60]).astype(int))

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        font_thickness = 2
        text_color = (209, 196, 233) 

        text_size, _ = cv2.getTextSize(str(Angle), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        cv2.rectangle(image, txt, (txt[0] + text_size[0], txt[1] - text_size[1]),(0, 0, 102), cv2.FILLED)

        cv2.putText(image, str(Angle), txt, font, font_scale, text_color, font_thickness, cv2.LINE_AA)

    return image


## Draw Transitions

In [64]:
def draw_filled_circle(image, center, color, fill_percentage):
    
    # Calculate the fill color based on the fill percentage
    fill_color = tuple(int(c * fill_percentage) for c in color)

    # Draw the outer circle
    outer_radius = 20
    cv2.circle(image, center, outer_radius, (255, 0, 127), thickness=3)

    # Draw the inner expanding circle
    inner_radius = int(outer_radius * fill_percentage)
    cv2.circle(image, center, inner_radius, fill_color, thickness= -1 )

## Comparsion

In [66]:
tolerance_deg = 10
Counter = 0
Msg =""
Num_Of_Pose_Completed = 0
Exercise_Name = "Trial"

target_angles_list = ['L_Shoulder', 'L_Elbow', 'L_Wrist']
pose_angles_list = [[56, 53, 170, 20, 170, 170, 170, 170, 180, 180, 170, 170, 90, 90],
                    [56, 53, 90, 20, 170, 170, 170, 170, 180, 180, 170, 170, 90, 90],
                    [56, 53, 90, 20, 90, 170, 170, 170, 180, 180, 170, 170, 90, 90]]


cap = cv2.VideoCapture(0)

while cap.isOpened():

    ret, frame = cap.read()
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    height, width, _ = image.shape
    # Process the image using the mediapipe pose model
    results = Pose.process(image)

    # Check if pose landmarks are present
    if results.pose_landmarks:
        L_Marks = results.pose_landmarks.landmark

        Drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            MP_Pose.POSE_CONNECTIONS,
            landmark_drawing_spec=Drawing_Spec_Landmarks,
            connection_drawing_spec=Drawing_Spec_Connections
        )
        

        # RT(Real Time) Angles list will hold angle for current frame
        RT_Angles = []
        # Correct Angles list will hold pose angles with which we want to compare our camera feedback
        Correct_Angles = []

        # Iterate through the list of target angles (angles of interest for each exercise)
        for angle_name in target_angles_list:

            # Val will hold the angle definition (i.e. Key - Value pair in Main_Angles dictionary)
            Val = Main_Angles[angle_name]
            # Calculate the angle and add it to RT_Angles
            rt_angle = Calculate_Angle([L_Marks[Val[0]].x, L_Marks[Val[0]].y],
                                        [L_Marks[Val[1]].x, L_Marks[Val[1]].y],
                                        [L_Marks[Val[2]].x, L_Marks[Val[2]].y])
            RT_Angles.append(rt_angle)

            correct_angle = pose_angles_list[Num_Of_Pose_Completed][Val[3]]
            Correct_Angles.append(correct_angle)


            # Calculate the fill percentage based on the similarity of the current angle to the target angle
            fill_percentage = 1.0 - (abs(rt_angle - correct_angle) / tolerance_deg)
            fill_percentage = max(0, min(1, fill_percentage))  # Ensure fill_percentage is in the range [0, 1]

            # Draw a filled circle to visualize the similarity
            center = tuple(np.multiply([L_Marks[Val[1]].x, L_Marks[Val[1]].y], [width, height]).astype(int))
            draw_filled_circle(image, center, (255, 0, 127), fill_percentage)


       
        #image = Draw_Angles(Counter, image, L_Marks, RT_Angles, Correct_Angles, tolerance_deg)
        Angle_Match = 0

        # Iterate through RT_Angles and Correct_Angles to tell how many angles are matched within a defined range (i.e. tolerance)
        for rt_angle, correct_angle in zip(RT_Angles, Correct_Angles):
            if abs(rt_angle - correct_angle) <= tolerance_deg:
                Angle_Match += 1

        # If all angles are matched , indicate that 1 pose is done . 
        if Angle_Match == len(RT_Angles):
            Angle_Match = 0
            Num_Of_Pose_Completed += 1

            # If all poses are done , increase exercise counter and start over.
            if Num_Of_Pose_Completed == len(pose_angles_list):
                Counter += 1
                Num_Of_Pose_Completed = 0
                Msg = f"{Counter} Times Elapsed."
            # Else , move to the next pose 
            else :
                Msg = f"Moving to Pose number {Num_Of_Pose_Completed + 1}."

        
        cv2.putText(image, Msg, (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 127), 2, cv2.LINE_AA)

        cv2.putText(image, str(Counter), (100,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 127), 2, cv2.LINE_AA)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        cv2.imshow('Phlex', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
